<a href="https://colab.research.google.com/github/GuiRibCarra/RNC-Gatos-Cachorros/blob/main/RNC_Gatos%26Cachorros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rede Neural Convolucional - Gatos e Cachorros

# Bibliotecas

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
from sklearn.metrics import accuracy_score

# Carregamento das Imagens

In [ ]:
path = '/content/drive/MyDrive/Cópia de dataset_gato_cachorro.zip'
zip_object = zipfile.ZipFile(file=path, mode="r")
zip_object.extractall("./")
zip_object.close()

# Construção: Base de Treinamento e de Teste

In [ ]:
gerador_treinamento = ImageDataGenerator(rescale = 1./255,
                                         rotation_range = 7,
                                         horizontal_flip = True,
                                         shear_range = 0.2,
                                         height_shift_range = 0.07,
                                         zoom_range = 0.2)

base_treinamento = gerador_treinamento.flow_from_directory('/content/dataset/training_set',
                                                           target_size = (64, 64),
                                                           batch_size = 32,
                                                           class_mode = 'binary')

gerador_teste = ImageDataGenerator(rescale = 1./255)

base_teste = gerador_teste.flow_from_directory('/content/dataset/test_set',
                                               target_size = (64, 64),
                                               batch_size = 32,
                                               class_mode = 'binary',
                                               shuffle=False)

Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# Construção e Treinamento da Rede Neural

In [ ]:
classificador = Sequential()
classificador.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(MaxPooling2D(pool_size = (2,2)))

classificador.add(Conv2D(32, (3,3), activation = 'relu'))
classificador.add(MaxPooling2D(pool_size = (2,2)))

classificador.add(Flatten())

classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dense(units = 1, activation = 'sigmoid'))

classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                      metrics = ['accuracy'])

In [ ]:
classificador.fit_generator(base_treinamento, epochs = 10, validation_data=base_teste)

<ipython-input-5-4a77c1193840>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classificador.fit_generator(base_treinamento, epochs = 10, validation_data=base_teste)


Epoch 1/10
125/125 [==============================] - 51s 394ms/step - loss: 0.6890 - accuracy: 0.5375 - val_loss: 0.6554 - val_accuracy: 0.6380
Epoch 2/10
125/125 [==============================] - 58s 464ms/step - loss: 0.6452 - accuracy: 0.6325 - val_loss: 0.6277 - val_accuracy: 0.6530
Epoch 3/10
125/125 [==============================] - 49s 393ms/step - loss: 0.6154 - accuracy: 0.6560 - val_loss: 0.6272 - val_accuracy: 0.6610
Epoch 4/10
125/125 [==============================] - 46s 369ms/step - loss: 0.5864 - accuracy: 0.6890 - val_loss: 0.6068 - val_accuracy: 0.6640
Epoch 5/10
125/125 [==============================] - 47s 374ms/step - loss: 0.5634 - accuracy: 0.7063 - val_loss: 0.5802 - val_accuracy: 0.7050
Epoch 6/10
125/125 [==============================] - 45s 360ms/step - loss: 0.5413 - accuracy: 0.7297 - val_loss: 0.5606 - val_accuracy: 0.7060
Epoch 7/10
125/125 [==============================] - 46s 365ms/step - loss: 0.5244 - accuracy: 0.7393 - val_loss: 0.5632 - val_ac

# Avaliação da Rede Neural

In [ ]:
previsoes = classificador.predict(base_teste)
previsoes = (previsoes > 0.5)
base_teste.class_indices

32/32 [==============================] - 6s 189ms/step


{'cachorro': 0, 'gato': 1}

In [ ]:
accuracy_score(previsoes, base_teste.classes)

0.748